**Imports**

In [1]:
import pandas as pd
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
import os
import numpy
from sklearn.preprocessing import LabelEncoder

**Reading files**

In [2]:
X_fileName = []
y_val = []

In [3]:
for fileName in sorted(glob.iglob('near-infrared/**/train_pose/**/**', recursive=False)):
    X_fileName.append(fileName)
    y_val.append(fileName.split("\\")[-2])

In [4]:
myDf = pd.DataFrame()
myDf['fileName'] = X_fileName
myDf['label'] = y_val

In [5]:
myLabelEncoder = LabelEncoder()
myDf['label_val'] = myLabelEncoder.fit_transform(myDf['label'])

**PreProcessing**

In [6]:
def pointremove(image):
    
    img=image.copy()
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    for i in range(1, nb_components):
        if sizes[i]<100 :
            img[output==i]=0
    return img

In [7]:
def maxmeanpreprop(path):
    img1 = cv2.imread(path)
    g_img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img = g_img.copy()
    ret1,th1 = cv2.threshold(g_img,40,255,cv2.THRESH_BINARY)
    th1=pointremove(th1)
    blur = cv2.GaussianBlur(th1,(5,5),0)
    ret2,th2 = cv2.threshold(blur,130,255,cv2.THRESH_OTSU)

    a_img=th2&img
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(a_img, connectivity=4)
    sizes = stats[:, -1]

    max_label = 0
    max_mean = 0

    for i in range(1, nb_components):
        mean=np.mean(a_img[output==i])
        if mean > max_mean and sizes[i]>1000 :
            max_label = i
            max_mean = mean

    img3 = np.zeros(output.shape)
    img3[output == max_label] = 255
    new_image = img3[stats[max_label][1]:stats[max_label][1]+stats[max_label][3],stats[max_label][0]:stats[max_label][0]+stats[max_label][2]]
    image_for_input = cv2.resize(new_image, (100, 100), interpolation = cv2.INTER_NEAREST)
    return image_for_input

**Creating my generator**

In [8]:
def my_generator(features, batch_size=128):
    # Create empty arrays to contain batch of features and labels#
    batch_features = np.zeros((batch_size, 100, 100, 1))
    batch_labels = np.zeros((batch_size))
    while True:
        for i in range(batch_size):
            # choose random index in features
            index = np.random.choice(len(features),1)
            batch_features[i] = maxmeanpreprop(features.loc[index[0]]['fileName']).reshape([100, 100, 1])
            batch_labels[i] = features.loc[index[0]]['label_val']
        yield batch_features, batch_labels

**Model** 

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
from keras.models import model_from_json
from keras.callbacks import EarlyStopping

Using TensorFlow backend.
C:\Users\akkir\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\akkir\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\akkir\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\akkir\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [10]:

def MyModel(input_shape):
    
    model = Sequential()

    model.add(Lambda(lambda x: x / 255 - 0.5, input_shape=input_shape))

    model.add(Conv2D(24, 5, 5, activation='relu', subsample=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(36, 5, 5, activation='relu', subsample=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(48, 5, 5, activation='relu', subsample=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(Dropout(0.1))

    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(Dropout(0.1))

    model.add(Flatten())

    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(10, activation='relu'))
    model.add(Dense(16, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  

    return model

In [11]:
myMod = MyModel([100, 100, 1])

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  import sys
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  del sys.path[0]
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  app.launch_new_instance()
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`


In [12]:
es = EarlyStopping(monitor='val_loss', patience=1)
myMod.fit_generator(my_generator(myDf), samples_per_epoch=100, nb_epoch=400,verbose=True, callbacks=[es])


C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., verbose=True, callbacks=[<keras.ca..., steps_per_epoch=100, epochs=400)`
  


Epoch 1/400
100/100 [==============================] - 171s 2s/step - loss: 2.6780 - accuracy: 0.1308
Epoch 2/400


C:\Users\akkir\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100/100 [==============================] - 167s 2s/step - loss: 2.3112 - accuracy: 0.2702
Epoch 3/400
100/100 [==============================] - 162s 2s/step - loss: 1.8239 - accuracy: 0.4188
Epoch 4/400
100/100 [==============================] - 165s 2s/step - loss: 1.4953 - accuracy: 0.5211
Epoch 5/400
100/100 [==============================] - 157s 2s/step - loss: 1.2410 - accuracy: 0.5992
Epoch 6/400
100/100 [==============================] - 157s 2s/step - loss: 1.0324 - accuracy: 0.6796
Epoch 7/400
100/100 [==============================] - 156s 2s/step - loss: 0.8963 - accuracy: 0.7242
Epoch 8/400
100/100 [==============================] - 155s 2s/step - loss: 0.7746 - accuracy: 0.7567
Epoch 9/400
100/100 [==============================] - 154s 2s/step - loss: 0.7159 - accuracy: 0.7782
Epoch 10/400
100/100 [==============================] - 162s 2s/step - loss: 0.6499 - accuracy: 0.7991
Epoch 11/400
100/100 [==============================] - 156s 2s/step - loss: 0.6220 - accurac

100/100 [==============================] - 171s 2s/step - loss: 0.1793 - accuracy: 0.9469
Epoch 82/400
100/100 [==============================] - 176s 2s/step - loss: 0.1766 - accuracy: 0.9466
Epoch 83/400
100/100 [==============================] - 161s 2s/step - loss: 0.1969 - accuracy: 0.9436
Epoch 84/400
100/100 [==============================] - 170s 2s/step - loss: 0.1918 - accuracy: 0.9423
Epoch 85/400
100/100 [==============================] - 184s 2s/step - loss: 0.1961 - accuracy: 0.9425
Epoch 86/400
100/100 [==============================] - 166s 2s/step - loss: 0.1836 - accuracy: 0.9455
Epoch 87/400
100/100 [==============================] - 157s 2s/step - loss: 0.1954 - accuracy: 0.9432
Epoch 88/400
100/100 [==============================] - 160s 2s/step - loss: 0.1787 - accuracy: 0.9453
Epoch 89/400
100/100 [==============================] - 151s 2s/step - loss: 0.1892 - accuracy: 0.9440
Epoch 90/400
100/100 [==============================] - 172s 2s/step - loss: 0.1723 - 

100/100 [==============================] - 160s 2s/step - loss: 0.1338 - accuracy: 0.9607
Epoch 161/400
100/100 [==============================] - 150s 1s/step - loss: 0.1302 - accuracy: 0.9604
Epoch 162/400
100/100 [==============================] - 155s 2s/step - loss: 0.1346 - accuracy: 0.9614
Epoch 163/400
100/100 [==============================] - 153s 2s/step - loss: 0.1276 - accuracy: 0.9623
Epoch 164/400
100/100 [==============================] - 146s 1s/step - loss: 0.1309 - accuracy: 0.9603
Epoch 165/400
100/100 [==============================] - 146s 1s/step - loss: 0.1345 - accuracy: 0.9578
Epoch 166/400
100/100 [==============================] - 148s 1s/step - loss: 0.1422 - accuracy: 0.9575
Epoch 167/400
100/100 [==============================] - 157s 2s/step - loss: 0.1267 - accuracy: 0.9645
Epoch 168/400
100/100 [==============================] - 146s 1s/step - loss: 0.1366 - accuracy: 0.9590
Epoch 169/400
100/100 [==============================] - 145s 1s/step - loss: 

100/100 [==============================] - 186s 2s/step - loss: 0.1108 - accuracy: 0.9664
Epoch 239/400
100/100 [==============================] - 188s 2s/step - loss: 0.1079 - accuracy: 0.9673
Epoch 240/400
100/100 [==============================] - 194s 2s/step - loss: 0.1229 - accuracy: 0.9641
Epoch 241/400
100/100 [==============================] - 187s 2s/step - loss: 0.1164 - accuracy: 0.9650
Epoch 242/400
100/100 [==============================] - 187s 2s/step - loss: 0.1124 - accuracy: 0.9655
Epoch 243/400
100/100 [==============================] - 185s 2s/step - loss: 0.1195 - accuracy: 0.9645
Epoch 244/400
100/100 [==============================] - 184s 2s/step - loss: 0.1175 - accuracy: 0.9666
Epoch 245/400
100/100 [==============================] - 192s 2s/step - loss: 0.1100 - accuracy: 0.9638
Epoch 246/400
100/100 [==============================] - 189s 2s/step - loss: 0.1075 - accuracy: 0.9659
Epoch 247/400
100/100 [==============================] - 187s 2s/step - loss: 

100/100 [==============================] - 144s 1s/step - loss: 0.0951 - accuracy: 0.9709
Epoch 317/400
100/100 [==============================] - 145s 1s/step - loss: 0.0941 - accuracy: 0.9698
Epoch 318/400
100/100 [==============================] - 149s 1s/step - loss: 0.1024 - accuracy: 0.9680
Epoch 319/400
100/100 [==============================] - 152s 2s/step - loss: 0.0891 - accuracy: 0.9721
Epoch 320/400
100/100 [==============================] - 153s 2s/step - loss: 0.1015 - accuracy: 0.9693
Epoch 321/400
100/100 [==============================] - 146s 1s/step - loss: 0.0964 - accuracy: 0.9713
Epoch 322/400
100/100 [==============================] - 146s 1s/step - loss: 0.0988 - accuracy: 0.9702
Epoch 323/400
100/100 [==============================] - 146s 1s/step - loss: 0.1031 - accuracy: 0.9684
Epoch 324/400
100/100 [==============================] - 146s 1s/step - loss: 0.1075 - accuracy: 0.9676
Epoch 325/400
100/100 [==============================] - 146s 1s/step - loss: 

100/100 [==============================] - 148s 1s/step - loss: 0.0905 - accuracy: 0.9727
Epoch 395/400
100/100 [==============================] - 149s 1s/step - loss: 0.0871 - accuracy: 0.9721
Epoch 396/400
100/100 [==============================] - 148s 1s/step - loss: 0.0893 - accuracy: 0.9730
Epoch 397/400
100/100 [==============================] - 154s 2s/step - loss: 0.0904 - accuracy: 0.9727
Epoch 398/400
100/100 [==============================] - 147s 1s/step - loss: 0.0912 - accuracy: 0.9730
Epoch 399/400
100/100 [==============================] - 146s 1s/step - loss: 0.1009 - accuracy: 0.9706
Epoch 400/400
100/100 [==============================] - 155s 2s/step - loss: 0.0932 - accuracy: 0.9720


In [19]:
X_file=[]
Y_va=[]
for fileName in sorted(glob.iglob('near-infrared/**/test_pose/**/**', recursive=False)):
    X_file.append(fileName)
    Y_va.append(fileName.split("\\")[-2])

In [20]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def testing(input_size):
    X=np.zeros((input_size,100,100,1))
    Y=np.zeros((input_size))
    
    Y1=myLabelEncoder.transform(Y_va)    
    k=-1
    for i in range(len(X_file)):
        k=k+1
        if (k==input_size):
            break
        image=maxmeanpreprop(X_file[i]).reshape(100,100,1)
        X[i]=image
        Y[i]=Y1[i]
        
    prediction = myMod.predict(X)
    compare_x=[]
    
    for i in range(input_size):
        result=np.where(prediction[i] == max(prediction[i]))
        if result[0] !=None:
            compare_x.append(result[0][0])
        else :
            compare_x.append(-1)        
    accuracy = accuracy_score(Y,compare_x)
    print('Accuracy: %f' % accuracy)
    f1 = f1_score(Y,compare_x,pos_label='positive',average='micro')
    print('F1 score: %f' % f1)
    print("Confusion Matrix :")
    print(confusion_matrix(compare_x,Y))
    return


In [21]:
testing(22579)

Accuracy: 0.954117
F1 score: 0.954117
Confusion Matrix :
[[1344    0    2    0    0    0    0    0    0    6    7    0    0    0
     0    0]
 [   0 1367    7    2    1    0   11    0   12    6    7    0    1    2
     6    0]
 [   3    2  906   49   22    0    6    0    0    0   31    4    0    0
     0    0]
 [   6    0   32 1254    4    0    0    0    0    6    4    1    4    0
     1    2]
 [   0    1    0    0 1182    5    0    0    0    1    7    0    0    0
     0    0]
 [   0    1    0    0   13 1205    2    0    0    1    1    0    6   35
     1    0]
 [   1   11    1    0    4    5 1203   19    1    7    0    0    0    0
     3    1]
 [   0    0    0    6    1    0    2 1325    2    2    0    1    0    1
     1    0]
 [   1    6    0    0    0    0   14    1 1306    4    1    1    0    0
     5    0]
 [   3    3    7    0    3    4    0    2    0 1348    0    2    0   11
     4    0]
 [   1    1   24    1   13    0   15    0    0    5 1959   64    0    0
     0    0]
 [  12  